## Apartado 1
Se debe implementar un sistema que permita recibir una expresión como entrada (será una expresión
formada solo por minúsculas y sin los signos de puntuación mencionados (los seis signos con los que
vamos a trabajar en este ejercicio y que serán siempre, el punto, la coma, los dos puntos, el punto y
coma, el signo de cierre de interrogación y de exclamación: `.,;:?!`), y la salida será la misma
expresión pero con los cambios correspondientes a la introducción de mayúsculas y signos de
puntuación indicados.
A un nivel alto de especificación podremos considerar que este método tiene esta signatura:
`string addPunctuationBasic(string)`

Es decir recibirá como entrada un string y devolverá como salida un string.
Como primera versión de esta función addPunctuationBasic se implementará un modelo que
simplemente cambia la primera letra por mayúscula y añade al final del string de entrada un punto.
Por ejemplo, al ejecutar

`addPunctuationBasic(“it can be a very complicated thing the ocean”)`
se obtendrá

>It can be a very complicated thing the ocean.

Agregó script para instalar las librerías requeridas por el proyecto. Dejo la opción de conda o directamente entorno python local..

In [ ]:
import sys

# Instalaciones directas en contexto conda
# !conda install --yes --prefix {sys.prefix} numpy
# !conda install --yes --prefix {sys.prefix} tqdm
# !conda install --yes --prefix {sys.prefix} sklearn

# Verificaciones para python
libraryList = !{sys.executable} -m pip list
if len(list(filter(lambda x: 'numpy ' in x, libraryList))) == 0:
    !{sys.executable} -m pip install numpy
if len(list(filter(lambda x: 'tqdm ' in x, libraryList))) == 0:
    !{sys.executable} -m pip install tqdm
if len(list(filter(lambda x: 'sklearn ' in x, libraryList))) == 0:
    !{sys.executable} -m pip install sklearn
    
import numpy as np
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1303 sha256=68bef46aab5ee27e3124ffe112dbfa28312ebb8ec5b339c20af7d4dda53da4d2
  Stored in directory: /home/germanlorenz/.cache/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


Realizo carga de datos para las tareas

In [ ]:
# Carga
with open('PunctuationTask.test.en') as f:
    PunctuationTaskTestEn = f.readlines()
with open('PunctuationTask.check.en') as f:
    PunctuationTaskCheckEn = f.readlines()
# Sin utf-8 me falla el corpus
with open('PunctuationTask.train.en', encoding='utf-8') as f:
    PunctuationTaskTrainEn = f.readlines()

# Verificacion
print(PunctuationTaskTestEn[0])
print(PunctuationTaskCheckEn[0])
print(PunctuationTaskTrainEn[0])

it can be a very complicated thing the ocean 

It can be a very complicated thing, the ocean. 

And it can be a very complicated thing, what human health is. 



Resolución de Apartado 1

In [ ]:
def addPunctuationBasic(line):
    # Caso especial
    if(line == ''):
        return ''
    # Convertimos el primer caracter de la oración a mayuscula
    first_letter = line[0].upper()
    return first_letter + line[1:] + '.'

Prueba con ejemplo de enunciado y corpus

In [ ]:
frase = 'it can be a very complicated thing the ocean'
print(addPunctuationBasic(frase))
print(addPunctuationBasic(PunctuationTaskTestEn[3]))

It can be a very complicated thing the ocean.
We made the ocean unhappy we made people very unhappy and we made them unhealthy 
.


## Apartado 2
Implementar la función verifyPunctuation con la siguiente signatura
`[(pos,err)] verifyPunctuation(string check, string test)`

Para realizar esta operación se llevará a cabo una tokenización de ambos strings. En este caso se
considerarán tokens todas las secuencias de letras, números y cualquier otro signo que no sea uno de
los seis indicados como signos de puntuación en este ejercicio (`.,;:?!`)
Es decir, esta función devolverá una lista de pares, donde cada par contendrá la posición (indicada
como índice en el string check) y el tipo de error. Los errores posibles son:
* ‘I’ → Insertion
* ‘D’ → Deletion
* ‘S’ → Substitution

Por ejemplo, consideremos que el string de referencia correcto (check) es
“Hello. What’s your name?”
La tokenización generará los siguientes 6 tokens de referencia
* Token 0: Hello
* Token 1: .
* Token 2: What’s
* Token 3: your
* Token 4: name
* Token 5: ?

Consideremos que nuestro algoritmo de puntuación (un caso hipotético para analizar el algoritmo de
verificación) genera la siguiente salida
>“Hello what’s your, name?”

Es decir los tokens generados en este caso son:
* Token 0: Hello
* Token 1: what’s
* Token 2: your
* Token 3: ,
* Token 4: name
* Token 5: ?

Debemos devolver la lista de cambios necesarios para convertir la cadena de tokens generados
(hipótesis) en la cadena de tokens correctos. Para ello, podemos inspirarnos en el algoritmo de la
distancia de Levenshtein. Es importante tener en cuenta que dadas dos cadenas A y B:
> Dist(A,B) == Dist(B,A)

Por lo que podemos abordar el problema tanto desde el punto de los cambios que hay que hacer para
llegar desde la hipótesis hasta el modelo correcto, o bien desde el modelo correco (referencia o check
en la terminología de este ejercicio) hasta la hipótesis generada por nuestro algoritmo de puntuación.
Podemos ver que respecto a nuestro string de referencia (check), el string de test ha ignorado (podemos
decir que borrado respecto al de referencia un ., por tanto tendríamos el error
>(‘D’,1)

En segundo lugar, el token 2 del string de referencia (check) se ha quedado mal en el string de test ya
que en lugar de “What’s” aparece “what’s”. Se trata por tanto de un error de substitución de una palabra
por otra (en este caso por un error de introducción de mayúsculas):
>(‘S’,2)

Y en tercer lugar, entre los tokens ‘your’ y ‘name’ del string correcto (check) se ha introducido un
nuevo token en el string de test, una coma en concreto, por tanto hay un error que calificaríamos como
>(‘I’,4)

Como se puede observar, todos los errores se posicionan (usan los índices) respecto al string correcto
de referencia (check).
Así pues el resultado de

`verifyPunctuation(“Hello. What’s your name?”, “Hello what’s your, name?”)`

sería

> [ (‘D’,1), (‘S’,2), (‘I’, 4) ]


Con la siguiente función tokenizo por palabras considerando tokens cuando estas se encuentran separadas por espacios o tokens reservados. De esta forma generar una lista de tokens para tratar las frases.

In [ ]:
# Funcion para tokenizar
def tokenizador(texto, tokens):
    
    # Verificacion strings con datos
    if len(texto) == 0: return 0
    texto_tokenizado = []
    
    # Generacion de tokens
    start = 0
    for i in range(len(texto)):
        # Espacios
        if texto[i] == ' ':
            texto_tokenizado.append(texto[start:i])
            start=i+1  
        # Analisis de prueba con tokens
        for j in range (len(tokens)):
            # tokens
            if texto[i] == tokens[j]:
                texto_tokenizado.append(texto[start:i])
                start=i
    return texto_tokenizado

Inspirandome en el algoritmo de Levenshtein reutilice la funcion referenciada aqui: https://python-course.eu/applications-python/levenshtein-distance.php para generar una matriz que me permita encontrar el recorrido optimo con las reglas minimales de insercion (insertion), borrado (deletion) y reemplazo (substitution). Al utilizar el algoritmo indicado obtuve una representación distinta a la recomendad en el enunciado como minimal en la salida de verifyPunctuation

In [ ]:
def iterative_levenshtein(s, t):
    """ 
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings 
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein 
        distance between the first i characters of s and the 
        first j characters of t
    """

    rows = len(s)+1
    cols = len(t)+1
    dist = [[0 for x in range(cols)] for x in range(rows)]

    # source prefixes can be transformed into empty strings 
    # by deletions:
    for i in range(1, rows):
        dist[i][0] = i

    # target prefixes can be created from an empty source string
    # by inserting the characters
    for i in range(1, cols):
        dist[0][i] = i
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = 1
            dist[row][col] = min(dist[row-1][col] + 1,      # deletion
                                 dist[row][col-1] + 1,      # insertion
                                 dist[row-1][col-1] + cost) # substitution
    return dist

Función para encontrar el recorrido mínimo con matriz.

In [ ]:
def minimal_operations(matriz_levenshtein, check_len, test_len):
  
    col = test_len
    row = check_len
    problemas = []
    while col > 1 and row > 1:
            
        minimo = min(matriz_levenshtein[row-1][col],      # deletion
                        matriz_levenshtein[row][col-1],   # insertion
                        matriz_levenshtein[row-1][col-1]) # substitution
        
        # Insertion
        if col == 1:
            while row > 1:
                problemas.append("I,"+str(row))
                row = row - 1
                
        # Insertion
        elif row == 1:
            while col > 1:
                problemas.append("D,"+str(row))
                col = col - 1
        
        # Diagonalizo        
        elif matriz_levenshtein[row-1][col-1] == minimo:
            if matriz_levenshtein[row-1][col-1] == matriz_levenshtein[row][col] - 1:
                # Substitution
                problemas.append("S,"+str(row))
            col = col - 1
            row = row - 1
        elif matriz_levenshtein[row-1][col] == minimo:
            # Deletion
            problemas.append("D,"+str(row))
            row = row - 1
        else:
            # Insertion
            problemas.append("I,"+str(row))
            col = col - 1
 
    return np.flip(problemas)   

Agregue un parámetro de reporte para poder visualizar la matriz y la posibilidad de no tokenizar asi procesa la información por carácter en vez de por token (palabra)

In [ ]:
def verifyPunctuation(check, test, tokenizar=True, extended_report=False):
    """ 
        Analizamos distancias según casos posibles combinando situaciones
        Agregue reporte para poder visualizar la matriz en las pruebas
        Agregue tokenizador para simplificar el procesamiento ya que el enunciado no es claro
    """ 
    # Caso especial
    if(check == '' or test == ''):
        return [('D', 0), ('S', 0), ('I', 0)]
    # Tokens reservados
    reserve_tokens = ['.',',',';',':','?','!']

    if tokenizar:
        # Tokenizamos input
        data_check = tokenizador(check, reserve_tokens)
        data_test = tokenizador(test, reserve_tokens)
    else:
        data_check = check
        data_test = test

    matriz_levenshtein = iterative_levenshtein(data_check, data_test)

    # Generar reporte
    if extended_report:
        # Impresion de verificacion, opcional
        print('DATOS CORRECTOS')
        print(data_check)
        print('DATOS A SER PROBADOS')
        print(data_test)
        
        # Impresion Matriz Levenshtein opcional
        rows = len(data_check)+1
        for r in range(rows):
            print(matriz_levenshtein[r])
            
    # Exportar camino
    # return minimal_operations(matriz_levenshtein, len(data_check), len(data_test))
    return minimal_operations(matriz_levenshtein, len(data_check), len(data_test))

In [ ]:
str1 = 'Hello. What’s your name?'
str2 = 'Hello what’s your, name?'

print(verifyPunctuation(str1,str2, tokenizar = False, extended_report = False ))
# ['D,6' 'S,8' 'I,18']
print(verifyPunctuation(str1,str2, tokenizar = True, extended_report = False))
# ['S,2' 'S,3' 'S,4']
print(PunctuationTaskCheckEn[3])
print(PunctuationTaskTestEn[3])
print(verifyPunctuation(PunctuationTaskCheckEn[3],PunctuationTaskTestEn[3], tokenizar = True))
print(verifyPunctuation(PunctuationTaskCheckEn[3],PunctuationTaskTestEn[3], tokenizar = False))

['S,2' 'S,3' 'S,4']
We made the ocean unhappy; we made people very unhappy, and we made them unhealthy. 

we made the ocean unhappy we made people very unhappy and we made them unhealthy 



## Apartado 3

Implementar una herramienta que permita recorrer todo el corpus de test y verificación. Es decir, iría
recorriendo una a una las líneas de cada fichero (que están alineadas), aplicaría sobre la frase de test el
algoritmo básico de puntuación (apartado 1: `addPunctuationBasic()` ) y a continuación
comprobaría si el resultado es o no correcto usando la función `verifyPunctuation()` del
apartado 2.

Obtener a continuación los valores relativos a `precisión`, exhaustividad (`recall`) y `F1` para el algoritmo
`addPunctuationBasic()` implementado en el apartado 1.

Consideraremos estos valores como el baseline, el modelo más básico de puntuación que podemos
realizar para estudiar posibles mejoras.

Funciones de `Accuracy`, `Precision`, `Recall` y `F1` basado en https://www.analyticslane.com/2019/10/09/numpy-basico-inicializacion-de-arrays-en-numpy/

Reporte general que voy a usar en los demas enunciados

In [ ]:
def reporteConMetricas(check, test, punctuationBasic = False):
    
    # Como los valores verdaderos son de check, asumo todos verdaderos ya que no tengo un algoritmo de prediccion
    y_true = np.ones(len(test))
    
    # Inicio mi vector de prediccion al cual voy a agregarle el resultado de mi verifyOuntuaction
    y_pred = np.zeros(len(test))
    
    # Variables
    tp = 0
    fp = 0
    fn = 0
    
    # Proceso
    for i in tqdm(range(len(test)),ncols = 100 , desc ="Verifico Metricas ...", disable = True):
        if punctuationBasic:
            test_proceced = addPunctuationBasic(test[i])
        else:
            test_proceced = test[i]
        
        analysis = len(verifyPunctuation(check[i], test_proceced)) > 0
        # Verificacion
        if not analysis:
            y_pred[i] = 1
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)     
    f1 = ((precision * recall)/(precision + recall)) * 2
    
    return (precision * 100, recall * 100, f1 * 100)


In [ ]:
precision, recall, f1 = reporteConMetricas(PunctuationTaskCheckEn, PunctuationTaskTestEn)
print("Metrica de implementacion Basica \n Precision: %s \n Recall: %s \n F1: %s \n" % (precision, recall, f1))

Metrica de implementacion Basica 
 Precision: 100.0 
 Recall: 0.2989848421638159 
 F1: 0.5961871750433275 



El resultado de las métricas indican que la herramienta desarrollada nos puede servir como una primer aproximación a la solución de problemas de puntuación aunque existen múltiples variantes con n-gramas, operación con ventanas, uso de distintos modelos en simultáneo o redes neuronales profundas que pueden brindar una significativa mejora en la predicción de la puntuación.

## Apartado 4

Utilizando el corpus de entrenamiento contenido en PunctuationTask.train.en construir un modelo de
lenguaje inspirado en la idea de 4-gramas. No es exactamente un 4-grama pero está basado en dicho
modelo.
El objetivo de este pseudo 4-grama será predecir si en la posición P de un string debemos introducir un
signo de puntuación (siempre estamos restringiendo el alcance a los seis signos de puntuación
considerados en este ejercicio), o si debemos cambiar la palabra en dicha posición P por mayúscula.
En última instancia, el 4-grama contendrá tuplas de la siguiente forma:


`(token1, token2, token3, operación)`

donde token1, token2 y token3 serán tokens cualesquiera incluidos en el corpus de entrenamiento. Por
tanto estos tokens podrán ser palabras o signos de puntuación, y ‘operación’ será una de las siguientes
operaciones:

`signo de puntuación (que podrá ser uno de los seis considerados: .,;;?!`

`mayúscula (que indica que la siguiente palabra se debe poner en mayúscula)`

`minúscula (que indica que la siguiente palabra deberá estar en minúscula)`

La operación se decide observando el fenómeno más común (frecuencia relativa) de las distintas
operaciones para cada tríada de tokens (token1 token2 token3).
Por ejemplo, podríamos detectar que para la tríada de tokens
`(‘by’, ‘the’, ‘way’)`
la operación más frecuente es insertar el signo de puntuación coma (,)
Una vez creado este modelo de lenguaje se debe implementar una segunda versión de la función que
añade signos de puntuación denominada
`string addPunctuation4gram(string)`

que recibirá como en el apartado 1 un string de entrada, y devolverá el nuevo string con los cambios
introducidos aplicando el modelo de lenguaje previamente entrenado con el 4-grama previamente
indicado.







In [ ]:
import re

token_regular_expression = r'[^.,:;?\s]+|[.,:;?]'
character_regular_expression = '([.,;:?!])'


def processTrainData(train_data):
    model_data = ' '.join(train_data).rstrip()
    tokens = re.findall(token_regular_expression, model_data)
    n_grams = []
    for i in range(0, len(tokens)-4):
        j = i+4
        n_grams.append(tokens[i:j])
    train_grams = np.array(n_grams)

    salida = {}
    
    for n_gram in train_grams:
        word_4ngram = n_gram[3]
        clave_palabra = ''.join(n_gram[0:3])
        next = ''
        if re.search(character_regular_expression, word_4ngram):
            next = 'C ' + word_4ngram
        elif word_4ngram[0].isupper():
            next = 'M'
        elif word_4ngram[0].isupper() != True:
            next = 'm'

        if clave_palabra in salida:
            salida[clave_palabra] = np.append(salida[clave_palabra], next)
        else:
            salida[clave_palabra] = np.array([next])

    
    for clave_palabra in salida:
        unique, counts = np.unique(salida[clave_palabra], return_counts=True)
        common = np.where(counts == max(counts))
        salida[clave_palabra] = unique[common]

    return salida

In [ ]:
def addPunctuation4gram(test_data, train_data):
    
    train_data = processTrainData(train_data)

    output = []
    
    for w in tqdm(range(len(test_data)),ncols = 100 , desc =" Entrenando ...", disable = True):
        tokens = re.findall(token_regular_expression, test_data[w])
        i = 0
        
        # Proceso principal
        while i < len(tokens)-4:
            sentence = tokens[i: i+4]
            items = sentence[0:3]
            key =  ''.join(items)
            if key in train_data:
                value = train_data[key][0]

                if 'C' in value:
                    target_token = tokens[i+3]
                    character =  value.split()[1]
                    if target_token != character:
                        tokens.insert(i+3,character)
                elif 'M' in value:
                    target_token = tokens[i+3]
                    if target_token[0].isupper() != True:
                        tokens[i+3] = target_token.capitalize()
                elif 'm' in value:
                    target_token = tokens[i+3]
                    if target_token[0].isupper():
                        tokens[i+3] = target_token.lower()

            i+=1
            
        transformation = ' '.join(tokens)
        # Agrego el trabajo de Apartado 1
        transformation = addPunctuationBasic(transformation)
        output.append(transformation)
    
    return output

# Ejecucion para reporte en Apartado 5
punctuation4gram_data = addPunctuation4gram(PunctuationTaskTestEn, PunctuationTaskTrainEn)

## Apartado 5

Aplicar el modelo de verificación implementado en el apartado 2, pero contrastando el corpus de
referencia con el resultado generado por el algoritmo de puntuación basado en 4-gramas del apartado 4.
A continuación obtener los valores de precisión, exhaustividad (recall) y F1 sobre estos nuevos
resultados y compararlos con los obtenidos en el apartado 3.
Este nuevo algoritmo de puntuación basado en 4-gramas, ¿mejora los resultados? Analiza si mejora o
empeora, ¿por qué puede ocurrir?

In [ ]:
#Original
precision, recall, f1 = reporteConMetricas(PunctuationTaskCheckEn, PunctuationTaskTestEn)
print("Metrica de implementacion Basica \n Precision: %s \n Recall: %s \n F1: %s \n" % (precision, recall, f1))

#4-grams
n_precision, n_recall, n_f1 = reporteConMetricas(PunctuationTaskCheckEn, punctuation4gram_data, punctuationBasic = False)
print("Metrica de 4-grams \n Precision: %s \n Recall: %s \n F1: %s \n" % (n_precision, n_recall, n_f1))

Metrica de implementacion Basica 
 Precision: 100.0 
 Recall: 0.2989848421638159 
 F1: 0.5961871750433275 

Metrica de 4-grams 
 Precision: 100.0 
 Recall: 0.32679738562091504 
 F1: 0.6514657980456027 



Los resultados que se pueden observar muestran que a pesar del entrenamiento con el corpus con 4-gramas no logra una mejora significativa dada la información con la que se evalúa aunque se sospecha que la implementación de tokenización puede ser la responsable del bajo rendimiento de ambos modelos.

## Apartado 6

Utilizando también ejemplos de TED talks, en un artículo de 2016, Ottokar Tilk y Tanel Alum ha
aplicado un modelo de redes recurrentes bidireccionales para esta misma tarea (restauración de signos
de puntuación en textos no segmentados).
El artículo donde lo describen se encuentra publicado en este enlace:

* https://www.isca-speech.org/archive/Interspeech_2016/pdfs/1517.PDF
* Bidirectional Recurrent Neural Network with Attention Mechanism forPunctuation Restoration (InterSpeech 2016).

El código correspondiente a su implementación se encuentra también disponible en github:

* https://github.com/ottokart/punctuator2

Los resultados de la evaluación para tres signos de puntuación concretos han sido estos (disponibles en
la dirección github anterior):

|PUNCTUATION | PRECISION | RECALL  | F-SCORE|
|:----|:----|:----|:----|
|,COMMA | 64.4 | 45.2 | 53.1|
|?QUESTIONMARK | 67.5 | 58.7 |  62.8|
|.PERIOD | 72.3 | 71.5 | 71.9|
|Overall | 68.9 | 58.1 | 63.1|

El objetivo de este apartado es estudiar la implementación que han hecho estos autores con una red
neuronal y adecuarla al problema que se ha planteado en este ejercicio.

En concreto, este apartado consistirá en la adecuación del modelo de Tilk y Alum al escenario de este
ejercicio. Es decir, aplicar el entrenamiento de la red propuesta por estos autores al corpus de
entrenamiento (PuntuationTask.train.en) y a continuación evaluar el resultado obtenido usando el
modelo de verificación implementado previamente.

Los resultados que obtienes para tu evaluación son similares a los publicados por estos autores.

Nota: Observar el modelo implementado por estos autores en el script error_calculator.py, como
contraste a vuestro algoritmo de verificación (apartado 2) y evaluación (apartados 3 y 5).




#### Análisis del modelo:


El modelo opera con texto no segmentado a través de un tipo de red neuronal recurrente. Entonces evita el trabajo de tokenizado en componentes más pequeños.

El modelo busca resolver específicamente problemas con comas, puntos, signos de exclamación, dos puntos, signos de interrogación los puntos y rayas.

El modelo BRNN permite utilizar contextos de tamaño no fijo antes y después de la posición actual de procesado.

Las capas recurrentes usan un tipo de unidad recurrente cerrada (GRU) para capturar dependencias de largo alcance en distintas unidades de tiempo. De esta forma las unidades tienen las mismas ventajas que las unidades LSTM pero siendo más sencillas en definitiva.

Este modelo se implementa con la librería theano y los pesos se van actualizando con un learning rate de 0.02.


#### Comparativa con el modelo realizado

El modelo propuesto en los apartados anteriores al ser comparado con esté arroja una métrica inferior dado que la capacidad de aprendizaje con el corpus de entrenamiento le brinda más flexibilidad a la hora de obtener el comportamiento deseado con los sets de prueba.

## Apartado 7

A partir del trabajo realizado y teniendo en cuenta otros enfoques disponibles en el ámbito de las
tecnologías del lenguaje, investiga qué otros enfoques o estrategias alternativas para esta misma tarea.
El objetivo de este apartado es que busques bibliografía relevante reciente sobre esta tarea, selecciones
uno o dos artículos y describas brevemente el enfoque y resultados obtenidos.



* A 43 Language Multilingual Punctuation Prediction Neural Network Model

El modelo se basa en BPE multi-idioma la cual es una técnica de compresión de datos que permite representar las palabras en unidades pequeñas reduciendo la dispersión de las palabras compartiendo unidades (subpalabras entre distintas palabras). Los investigadores demuestran que un modelo multilingüe que considere la compatibilidad entre varios idiomas, a diferencia de los modelos monolingües tradicionales puede mejorar la precisión. En el artículo demuestran que un solo modelo multilingüe basado en BPE puede lograr un rendimiento similar o superior a los modelos monolingües por separado basados en palabras. EL modelo logra un puntaje F1 promedio del 80,2% analizando noticias mientras que en el reconocimiento de voz fuera del dominio logra un puntaje de 73,3% en F1.


EL modelo presenta los siguientes resultados al combinar idiomas.

Español intersectado con la unión de Francés e italiano en un 93.2%.
Francés  intersectado con la unión de Italiano y Español en un 95,4%.
Italiano intersectado con la unión de Francés y ESpañol en un 95,8%.

ref: https://indico2.conference4me.psnc.pl/event/35/contributions/2972/attachments/610/641/Mon-3-1-9.pdf


* Token-Level Supervised Contrastive Learning for Punctuation Restoration

En este trabajo se unifican varias estrategias de codificación existentes para predecir la puntuación utilizando múltiples predicciones en cada palabra en diferentes ventanas. Los investigadores demuestran que se pueden lograr mejoras significativas al optimizar estrategias luego de entrenar un modelo aumentando el tiempo de respuesta sin necesidad de re-entrenamiento.

El modelo propuesto logra una mejora sustancial cuando hay poco o ningún contexto en el right-side al entrenar. La estrategia que utilizan es usar un token [PUNCT] que informa al modelo de la posición de la puntuación que se va a predecir. Los textos de entrada se tokenizan y la predicción se da evaluando múltiples posibles lugares donde el token de puntuación debiera ser insertado. Además se optimiza el tamaño de ventana y la longitud de zancada al momento de la clasificación. Por último cuando se debe elegir entre un modelo u otro existe una compensación entre la latencia y la precisión entre modelos. De esta forma se utiliza un modelo de clasificación cuando no hay un contexto futuro mientras que un modelo de etiquetado es usado cuando hay contexto del lado derecho a predecir.

Los valores generales que ambos modelos logran en F1 para enfoques de clasificación y etiquetado  para la predicción de puntuación en tiempo real con anticipación (lookahead) de hasta 4 es de 76,3% en TAgging y 73,9 en clasificación, mientras que sin anticipación l = 0 el modelo de Tagging logra 42,1% y el de clasificación 47,3%

ref: https://arxiv.org/pdf/2112.08098v2.pdf